In [1]:
import os


IMAGES_DIR = "/content/drive/MyDrive/Plate_Recognition/turkish_plate_data/images"

txt_silinen_adet = 0
silme_basladi = False

print(f"'{IMAGES_DIR}' dizinindeki tüm .txt dosyaları aranıyor ve siliniyor...")

if os.path.exists(IMAGES_DIR):
    silme_basladi = True


    for root, _, files in os.walk(IMAGES_DIR):
        for file in files:
            # Dosyanın .txt uzantılı olup olmadığını kontrol et
            if file.endswith('.txt'):
                file_path = os.path.join(root, file)

                # Dosyayı sil
                os.remove(file_path)
                txt_silinen_adet += 1

    if txt_silinen_adet > 0:
        print(f"\n✅ Başarıyla tamamlandı: {txt_silinen_adet} adet TXT dosyası silindi.")
    else:
        print("\nℹ️ Silinecek .txt dosyası bulunamadı. Klasör zaten temizlenmiş olabilir.")
else:
    print(f"\n❌ HATA: {IMAGES_DIR} yolu bulunamadı. Lütfen kopyalama işleminin başarılı olduğundan emin olun.")


if silme_basladi:
    print("\nKontrol (Images Klasörü İlk 5 Öğesi):")
    try:
        first_level_items = os.listdir(IMAGES_DIR)
        print(first_level_items[:5])
    except Exception as e:
        print(f"Klasör içeriği listelenirken hata: {e}")

'/content/drive/MyDrive/Plate_Recognition/turkish_plate_data/images' dizinindeki tüm .txt dosyaları aranıyor ve siliniyor...

✅ Başarıyla tamamlandı: 1955 adet TXT dosyası silindi.

Kontrol (Images Klasörü İlk 5 Öğesi):
['1495.jpg', '1307.jpg', '767.jpg', '1402.jpg', '1803.jpg']


In [4]:
import os
import shutil
import random
from tqdm import tqdm

# ====================================================================
# SETTINGS
# ====================================================================

SOURCE_ROOT_DIR = "/content/drive/MyDrive/Plate_Recognition/turkish_plate_data"

DEST_ROOT_DIR = "/content/drive/MyDrive/Plate_Recognition/turkish_plate_data_splitted"

TRAIN_RATIO = 80
VAL_RATIO = 10
TEST_RATIO = 10

# ====================================================================
# FUNCTION
# ====================================================================

def split_yolo_dataset(source_dir, dest_dir, train_ratio, val_ratio, test_ratio):

    if train_ratio + val_ratio + test_ratio != 100:
        raise ValueError("Train, Validation ve Test oranlarının toplamı 100 olmalıdır!")

    random.seed(42)

    IMAGES_SOURCE = os.path.join(source_dir, 'images')
    LABELS_SOURCE = os.path.join(source_dir, 'label')

    if not os.path.isdir(IMAGES_SOURCE) or not os.path.isdir(LABELS_SOURCE):
        print(f"Hata: Giriş dizinlerinde 'images' ve 'labels' klasörleri bulunamadı. Lütfen '{source_dir}' yolunu kontrol edin.")
        return

    if os.path.exists(dest_dir):
        shutil.rmtree(dest_dir)
    os.makedirs(dest_dir)

    subsets = {
        'train': train_ratio / 100.0,
        'val': val_ratio / 100.0,
        'test': test_ratio / 100.0
    }

    for subset in subsets.keys():
        os.makedirs(os.path.join(dest_dir, subset, 'images'))
        os.makedirs(os.path.join(dest_dir, subset, 'labels'))

    print(f"Hedef klasörler oluşturuldu ve temizlendi: {dest_dir}")

    image_files = [f for f in os.listdir(IMAGES_SOURCE) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    random.shuffle(image_files)
    total_files = len(image_files)

    print(f"Toplamda işlenecek dosya sayısı: {total_files}")

    train_count = int(total_files * subsets['train'])
    val_count = int(total_files * subsets['val'])

    train_files = image_files[:train_count]
    val_files = image_files[train_count:train_count + val_count]
    test_files = image_files[train_count + val_count:]


    split_map = {'train': train_files, 'val': val_files, 'test': test_files}

    for subset_name, files_list in split_map.items():
        print(f"\n{subset_name.upper()} setine {len(files_list)} dosya kopyalanıyor...")

        for img_file in tqdm(files_list):

            base_name, _ = os.path.splitext(img_file)
            label_file = base_name + '.txt'


            img_src = os.path.join(IMAGES_SOURCE, img_file)
            label_src = os.path.join(LABELS_SOURCE, label_file)

            img_dst_dir = os.path.join(dest_dir, subset_name, 'images')
            label_dst_dir = os.path.join(dest_dir, subset_name, 'labels')


            shutil.copy2(img_src, img_dst_dir)


            if os.path.exists(label_src):
                 shutil.copy2(label_src, label_dst_dir)



    print("\n\n🎉 Tüm veri seti başarıyla train, val ve test setlerine ayrıldı!")
    print(f"Sonuçlar burada: {dest_dir}")

# ====================================================================
# RUN CODE
# ====================================================================

try:
    split_yolo_dataset(
        source_dir=SOURCE_ROOT_DIR,
        dest_dir=DEST_ROOT_DIR,
        train_ratio=TRAIN_RATIO,
        val_ratio=VAL_RATIO,
        test_ratio=TEST_RATIO
    )
except ValueError as e:
    print(f"Hata: {e}")
except Exception as e:
    print(f"Beklenmeyen bir hata oluştu: {e}")

Hedef klasörler oluşturuldu ve temizlendi: /content/drive/MyDrive/Plate_Recognition/turkish_plate_data_splitted
Toplamda işlenecek dosya sayısı: 1955

TRAIN setine 1564 dosya kopyalanıyor...


100%|██████████| 1564/1564 [07:56<00:00,  3.28it/s]



VAL setine 195 dosya kopyalanıyor...


100%|██████████| 195/195 [00:51<00:00,  3.76it/s]



TEST setine 196 dosya kopyalanıyor...


100%|██████████| 196/196 [00:47<00:00,  4.09it/s]



🎉 Tüm veri seti başarıyla train, val ve test setlerine ayrıldı!
Sonuçlar burada: /content/drive/MyDrive/Plate_Recognition/turkish_plate_data_splitted
